In [1]:
import pandas as pd
import numpy as np

In [2]:
datas = pd.read_csv('P3_GrantExport.csv',sep=';')

In [3]:
data_final = datas[[0,7,11,12,13]]
data_final.head()

,"﻿""Project Number""",University,Start Date,End Date,Approved Amount
0,1,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,4,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,5,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,6,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,7,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


In [4]:
data_final=data_final.rename(columns={'\ufeff"Project Number\"':'Project Number'})
data_final.head()

,Project Number,University,Start Date,End Date,Approved Amount
0,1,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,4,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,5,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,6,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,7,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


In [5]:
data_final.set_index('Project Number')
data_final.head()

,Project Number,University,Start Date,End Date,Approved Amount
0,1,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,4,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,5,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,6,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,7,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


Some data need to be replaced with NaN, as they do not convey any information regarding the location of the grant that was assigned. These are the following :
- 'Nicht zuteilbar - NA' -> Unavailable data
- 'NPO (Biblioth., Museen, Verwalt.) - NPO' -> No informations
- 'Weitere Institute - FINST' -> Other institutes
- 'Firmen/Privatwirtschaft - FP' -> Private institutions

In [12]:
data_final = data_final.replace('Nicht zuteilbar - NA',np.nan)
data_final = data_final.replace('NPO (Biblioth., Museen, Verwalt.) - NPO',np.nan)
data_final = data_final.replace('Weitere Institute - FINST',np.nan)
data_final = data_final.replace('Firmen/Privatwirtschaft - FP',np.nan)

data_uni = data_final['University'].unique()[1:]

data_uni

array(['Université de Genève - GE', 'Universität Basel - BS',
       'Université de Fribourg - FR', 'Universität Zürich - ZH',
       'Université de Lausanne - LA', 'Universität Bern - BE',
       'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL',
       'Université de Neuchâtel - NE', 'ETH Zürich - ETHZ',
       'Inst. de Hautes Etudes Internat. et du Dév - IHEID',
       'Universität St. Gallen - SG',
       'Pädagogische Hochschule Graubünden - PHGR', 'EPF Lausanne - EPFL',
       'Pädagogische Hochschule Zürich - PHZFH', 'Universität Luzern - LU',
       'Schweiz. Institut für Kunstwissenschaft - SIK-ISEA',
       'SUP della Svizzera italiana - SUPSI',
       'HES de Suisse occidentale - HES-SO',
       'Robert Walser-Stiftung Bern - RWS', 'Paul Scherrer Institut - PSI',
       'Pädagogische Hochschule St. Gallen - PHSG',
       'Eidg. Anstalt für Wasserversorgung - EAWAG',
       'Eidg. Material und Prüfungsanstalt - EMPA',
       'Physikal.-Meteorolog. Observatorium Davos - PM

In [13]:
test = data_uni[12].split('-')
print(test)

['EPF Lausanne ', ' EPFL']


In [14]:
#A INSTALLER AVEC pip install geopy

In [15]:
from geopy.geocoders import GeoNames
geolocator = GeoNames(country_bias='Switzerland',username='test_056')
print(geolocator.geocode('EPFL'))

Ecole Polytechnique Fédérale de Lausanne, VD, CH


In [16]:
def extract_canton(data_string):
    """
        Fetchs from the web the location associated to a string. We use Nominatim at the moment. 
        The format of the output string (if it is not None) has the canton in the 6th output before the end.
        The canton is written in all the languages spoken in it, and we hence split and keep the name that comes first.
        @param data_string : the data from which we want to determine the canton
        @return canton : the name of the canton associated to the input data_string
    """
    geolocator = Nominatim(country_bias='Switzerland')
    location = geolocator.geocode(data_string)
    split_loc = str(location).split(', ')
    if (split_loc is not None) and (split_loc[-1] =='Svizra'):
        canton = str(location).split(', ')
        canton = canton[len(canton)-6]
        return canton.split(' - ')[0]   
    else:
        return None

In [19]:
def extract_canton_from_uni(data_uni):
    """
        Extracts to which canton belongs a university and stored it into a json file (folder data)
        @param data_uni :           an array of strings describing universities or other institutions.
        @return university_dict :   a dict which associate to each university a canton if it was found
                                    or a None if nothing was found.
    """
    if(os.path.isfile('data/uni_canton_dict.json')):
        print('Loading the data from json file')
        with open('data/uni_canton_dict.json') as f:
            university_dict = json.load(f)
    else:
        print('Fetching the locations from th web')
        university_dict =dict()
        for index,uni in enumerate(data_uni):
            # Splits the university string at the '-', which usually corresponds to the separation between the full name
            # and the acronym of the institution.
            uni_name_split = uni.split('-')
            extract_value = extract_canton(uni_name_split[0])
            if (extract_value is None) and (len(uni_name_split) > 1):
                # If the canton was not found in the name, then we try to extract it from the acronym string if it exists
                #print(uni_name_split,' ',extract_value,'\n')
                extract_value = extract_canton(uni_name_split[1])
                #print(extract_value)
            university_dict[data_uni[index]] = extract_value
        with open('data/uni_canton_dict.json', 'w') as f:
            json.dump(university_dict, f)
    return university_dict
print(extract_canton_from_uni(data_uni))
        

Loading the data from json file
{'Eidg. Anstalt für Wasserversorgung - EAWAG': 'Zürich', 'Kantonsspital St. Gallen - KSPSG': 'Sankt Gallen', 'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': 'Basel-Landschaft', 'Haute école pédagogique BE, JU, NE - HEPBEJUNE': None, 'Eidg. Hochschulinstitut für Berufsbildung - EHB': None, 'Berner Fachhochschule - BFH': 'Bern', 'Pädagogische Hochschule Luzern - PHLU': 'Luzern', 'Institut für Kulturforschung Graubünden - IKG': None, 'Université de Lausanne - LA': 'Vaud', 'Swiss Center for Electronics and Microtech. - CSEM': None, 'Inst. de Hautes Etudes Internat. et du Dév - IHEID': None, 'Swiss Institute of Bioinformatics - SIB': None, 'Haute école pédagogique fribourgeoise - HEPFR': None, 'Schweiz. Institut für Kunstwissenschaft - SIK-ISEA': None, 'Eidg. Material und Prüfungsanstalt - EMPA': None, 'Forschungsanstalten Agroscope - AGS': None, 'Forschungskommission SAGW': None, 'ETH Zürich - ETHZ': 'Zürich', 'Friedrich Miescher Institute - FMI': None, '

In [ ]:
def 

TODO : 
1. We sort by university
    - if nan : go to institution
    - if none : go to institution
    - if found : stop
2. For the institution :


